In [0]:
%sh

# We switch to the root directory of our driver node
cd /
pwd
# We create a directory called input_wb_data and move into it
mkdir input_wb_data
cd input_wb_data
# We create a variable for the URL from which we'll fetch the data file
DATA_URL='https://databank.worldbank.org/data/download/WDI_csv.zip'
# We use the wget command to fetch the data
wget -O world_development_indicators.zip $DATA_URL
# We unzip the compressed file we received to extract the CSV files
unzip world_development_indicators.zip
rm world_development_indicators.zip

/
--2022-07-09 12:44:09--  https://databank.worldbank.org/data/download/WDI_csv.zip
Resolving databank.worldbank.org (databank.worldbank.org)... 192.86.98.102
Connecting to databank.worldbank.org (databank.worldbank.org)|192.86.98.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67657402 (65M) [application/x-zip-compressed]
Saving to: ‘world_development_indicators.zip’

     0K .......... .......... .......... .......... ..........  0%  233K 4m43s
    50K .......... .......... .......... .......... ..........  0%  349K 3m56s
   100K .......... .......... .......... .......... ..........  0%  702K 3m8s
   150K .......... .......... .......... .......... ..........  0%  692K 2m45s
   200K .......... .......... .......... .......... ..........  0%  696K 2m31s
   250K .......... .......... .......... .......... ..........  0%  699K 2m21s
   300K .......... .......... .......... .......... ..........  0%  707K 2m14s
   350K .......... .......... .......... .....

In [0]:
%sh

# We switch to the root directory of our driver node
cd /
cd input_wb_data
ls


WDICountry-Series.csv
WDICountry.csv
WDIData.csv
WDIFootNote.csv
WDISeries-Time.csv
WDISeries.csv


In [0]:
for name in [
    'WDICountry-Series.csv', 
    'WDICountry.csv',
    'WDIData.csv',
    'WDIFootNote.csv',
    'WDISeries-Time.csv',
    'WDISeries.csv',
]:
    dbutils.fs.cp(f'file:/input_wb_data/{name}', f'/datalake/raw/world_development_indicators/date=20220324/{name}')

In [0]:
display(dbutils.fs.ls('/datalake/raw/world_development_indicators/date=20220324'))

path,name,size,modificationTime
dbfs:/datalake/raw/world_development_indicators/date=20220324/WDICountry-Series.csv,WDICountry-Series.csv,1023586,1657370844000
dbfs:/datalake/raw/world_development_indicators/date=20220324/WDICountry.csv,WDICountry.csv,157138,1657370845000
dbfs:/datalake/raw/world_development_indicators/date=20220324/WDIData.csv,WDIData.csv,208343996,1657370851000
dbfs:/datalake/raw/world_development_indicators/date=20220324/WDIFootNote.csv,WDIFootNote.csv,67290069,1657370858000
dbfs:/datalake/raw/world_development_indicators/date=20220324/WDISeries-Time.csv,WDISeries-Time.csv,51756,1657370859000
dbfs:/datalake/raw/world_development_indicators/date=20220324/WDISeries.csv,WDISeries.csv,3769712,1657370860000


In [0]:
%sh
apt-get install jq
# We switch to the root directory of our driver node
cd /
# We create a directory called eea_input_data and move into it
mkdir eea_input_data
cd eea_input_data/
# We create a set of variables for the data URL
BASE_URL="https://discodata.eea.europa.eu"
TABLE="CO2Emission.latest.co2cars"
ORDER="ORDER%20BY%20%27Enedc%20(g%2Fkm)%27%20DESC"
# We go through the three years via a for loop
for YEAR in 2017 2018 2019
do
     CONDITION="year%20%3D%20$YEAR"
     SQL_QUERY="SELECT%20*%20FROM%20$TABLE%20WHERE%20$CONDITION%20$ORDER"
     FULL_URL="$BASE_URL/sql?query=$SQL_QUERY&p=1&nrOfHits=100000"
     curl $FULL_URL | jq '.results' > co2_emissions_passenger_cars_$YEAR.json
done

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  jq
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 50.2 kB of archives.
After this operation, 99.3 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 jq amd64 1.6-1ubuntu0.20.04.1 [50.2 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 50.2 kB in 0s (107 kB/s)
Selecting previously unselected package jq.
(Reading database ... 
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Rea

In [0]:
display(dbutils.fs.ls("file:/eea_input_data"))

path,name,size,modificationTime
file:/eea_input_data/co2_emissions_passenger_cars_2017.json,co2_emissions_passenger_cars_2017.json,82026594,1657371183392
file:/eea_input_data/co2_emissions_passenger_cars_2019.json,co2_emissions_passenger_cars_2019.json,83522920,1657371216996
file:/eea_input_data/co2_emissions_passenger_cars_2018.json,co2_emissions_passenger_cars_2018.json,82271106,1657371200212


In [0]:
%sh ls /eea_input_data

co2_emissions_passenger_cars_2017.json
co2_emissions_passenger_cars_2018.json
co2_emissions_passenger_cars_2019.json


In [0]:
for filename in [
    'co2_emissions_passenger_cars_2017.json',
    'co2_emissions_passenger_cars_2018.json',
    'co2_emissions_passenger_cars_2019.json',
]:
    year = filename.split('.')[0].split('_')[-1]
    dbutils.fs.cp(f'file:/eea_input_data/{filename}', f'/datalake/raw/co2_passenger_cars_emissions/year={year}')

In [0]:
display(dbutils.fs.ls('/datalake/raw/co2_passenger_cars_emissions/'))

path,name,size,modificationTime
dbfs:/datalake/raw/co2_passenger_cars_emissions/year=2017,year=2017,82026594,1657371422000
dbfs:/datalake/raw/co2_passenger_cars_emissions/year=2018,year=2018,82271106,1657371425000
dbfs:/datalake/raw/co2_passenger_cars_emissions/year=2019,year=2019,83522920,1657371428000


In [0]:
df = spark.read.option("header", True).csv('/datalake/raw/world_development_indicators/date=20220324/WDIData.csv')

In [0]:
df.printSchema()

root
 |-- Country Name: string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- Indicator Name: string (nullable = true)
 |-- Indicator Code: string (nullable = true)
 |-- 1960: string (nullable = true)
 |-- 1961: string (nullable = true)
 |-- 1962: string (nullable = true)
 |-- 1963: string (nullable = true)
 |-- 1964: string (nullable = true)
 |-- 1965: string (nullable = true)
 |-- 1966: string (nullable = true)
 |-- 1967: string (nullable = true)
 |-- 1968: string (nullable = true)
 |-- 1969: string (nullable = true)
 |-- 1970: string (nullable = true)
 |-- 1971: string (nullable = true)
 |-- 1972: string (nullable = true)
 |-- 1973: string (nullable = true)
 |-- 1974: string (nullable = true)
 |-- 1975: string (nullable = true)
 |-- 1976: string (nullable = true)
 |-- 1977: string (nullable = true)
 |-- 1978: string (nullable = true)
 |-- 1979: string (nullable = true)
 |-- 1980: string (nullable = true)
 |-- 1981: string (nullable = true)
 |-- 1982: string (null

In [0]:
total_records = df.count()
print(f'Total record for World Development Indicators data is {total_records} records')

Total record for World Development Indicators data is 383572 records


In [0]:
display(df.describe())

summary,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,_c66
count,383572,383572,383572,383572,36838,41776,43607,43844,44364,46567,46593,47945,47922,48707,66380,73461,76168,76092,77579,81272,83597,87116,87031,87907,92348,94687,95993,95936,96613,97957,98735,99963,98818,100333,125938,133367,138299,138818,141100,148981,148862,151243,152478,157967,184995,178488,183876,183795,188940,200827,200681,205369,203935,205239,218676,211880,215757,210707,217719,214441,213076,211018,206570,190581,152073,62283,0
mean,null,null,null,null,4.593238561379317E11,4.3359180445857117E11,4.391557634501739E11,4.556668550912128E11,4.8876853955172345E11,5.334058536123897E11,5.795181613195906E11,6.241948150127189E11,7.125984070730144E11,7.8781199866958E11,6.692400237914834E11,6.94165584058924E11,7.682375574195485E11,9.033858641234233E11,1.2030546552630193E12,1.2206079589894937E12,1.3541243937439841E12,1.29264604011477E12,1.1316508255886438E12,1.0747818254838644E12,8.486550362802478E11,8.149053240055345E11,8.891958983016543E11,1.123818742343599E12,1.1451592901250332E12,1.2381867313954065E12,1.1529725097220928E12,1.1512989382142148E12,9.033361024584807E11,9.719880548190267E11,9.934485547765732E11,9.89993896295386E11,9.945339254985319E11,1.0917591217192368E12,1.151323165826209E12,1.145397068513021E12,1.1240528142958867E12,1.122612216760289E12,1.0226056259525144E12,1.0048626468627539E12,9.120886794023835E11,9.865156484540457E11,1.1746222209276196E12,1.1499719270114429E12,1.307210328536657E12,1.322258039344117E12,1.4716055481868262E12,1.721957639370875E12,1.9070645275333616E12,1.9661997539337864E12,2.1565830763375454E12,2.4749169493732104E12,2.539233841338145E12,2.8499346510931914E12,2.988078663402137E12,3.0935285723457725E12,3.4147046531150933E12,3.584272818940901E12,4.058773215838249E12,4.438757995178797E12,6.223449634998403E12,7.329840666813507E12,null
stddev,null,null,null,null,2.035263141287935E13,2.072121682093602E13,2.1601228266991152E13,2.281363320071661E13,2.4611452688670902E13,2.7846680067818336E13,3.082112242236373E13,3.393026954406537E13,3.875546076805367E13,4.3983739060633875E13,4.1795037568577055E13,4.566795652264564E13,5.1760816999080266E13,5.9501174437547305E13,8.121972628174175E13,8.155695687081106E13,9.273949076499119E13,8.856870869211372E13,7.381412799449489E13,6.9429798601627766E13,5.158730771137211E13,4.834578878351882E13,5.419209951637073E13,6.747607048861583E13,7.048576116260164E13,8.329800586619375E13,7.737511744612942E13,7.058743098321492E13,4.715984280652132E13,5.072297377507371E13,5.8296824119383305E13,5.814215193083209E13,5.880066366547539E13,6.486499623014481E13,6.9441701185588914E13,6.9191922515243734E13,6.085007643732241E13,5.917300493257964E13,5.6283288958083836E13,5.87445109410099E13,6.022829840015572E13,6.674958782875926E13,6.535194214318123E13,7.721526769251692E13,7.528203620754131E13,7.95634496289694E13,8.391445265848003E13,9.078360738663608E13,9.443400243874123E13,9.61821837605795E13,1.0576971310300602E14,1.1686117020066036E14,1.1678404079222256E14,1.2861644869321081E14,1.3768163986921294E14,1.413329039329102E14,1.577114623993154E14,1.688843379242037E14,1.939602415328235E14,2.1901257586623628E14,3.070496681977441E14,2.3322919135616056E14,null
min,Afghanistan,ABW,ARI treatment (% of children under 5 taken to a health provider),AG.AGR.TRAC.NO,-0.003308390301,-0.0017284022427333,-0.000155636363545455,-0.000243272726363636,-0.000112838000000001,-0.000641454544454546,-0.000554181818454545,-0.000876727273454545,-0.0002550001,-0.000425453699999996,-0.0002970001,-0.00064314309999999,-0.000224618999999992,-0.000459999996666664,-0.000896666666666673,-0.000168580785369902,-0.000101563499999457,-0.000157833,-0.00013113,-0.00

In [0]:
#display(df)

In [0]:

df2 = spark.read.option("multiline", "true").json('/datalake/raw/co2_passenger_cars_emissions')

In [0]:
df2.printSchema()

root
 |-- At1 (mm): long (nullable = true)
 |-- At2 (mm): long (nullable = true)
 |-- Cn: string (nullable = true)
 |-- Cr: string (nullable = true)
 |-- Ct: string (nullable = true)
 |-- De: double (nullable = true)
 |-- E (g/km): string (nullable = true)
 |-- Enedc (g/km): long (nullable = true)
 |-- Er (g/km): string (nullable = true)
 |-- Ernedc (g/km): double (nullable = true)
 |-- Erwltp (g/km): double (nullable = true)
 |-- Ewltp (g/km): long (nullable = true)
 |-- Fm: string (nullable = true)
 |-- Ft: string (nullable = true)
 |-- ID: long (nullable = true)
 |-- It: string (nullable = true)
 |-- MMS: string (nullable = true)
 |-- MS: string (nullable = true)
 |-- Man: string (nullable = true)
 |-- Mh: string (nullable = true)
 |-- Mk: string (nullable = true)
 |-- Mp: string (nullable = true)
 |-- Mt: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- T: string (nullable = true)
 |-- TAN: string (nullable = true)
 |-- VFN: string (nullable = true)
 |-- Va: string

In [0]:
total_records = df2.count()
print(f'Total record for co2_emissions_passenger_cars is {total_records} records')

Total record for co2_emissions_passenger_cars is 300000 records


In [0]:
display(df2.describe())

summary,At1 (mm),At2 (mm),Cn,Cr,Ct,De,E (g/km),Enedc (g/km),Er (g/km),Ernedc (g/km),Erwltp (g/km),Ewltp (g/km),Fm,Ft,ID,It,MMS,MS,Man,Mh,Mk,Mp,Mt,Status,T,TAN,VFN,Va,Ve,Vf,W (mm),Zr,ec (cm3),ep (KW),m (kg),r,version_file,year,z (Wh/km)
count,283792,276259,299985,299094,298302,10010,0,300000,0,7154,2812,109308,299993,300000,300000,275337,299262,299996,300000,300000,299782,300000,109757,300000,299974,299872,285543,299901,299844,16342,286690,0,299975,248548,299434,300000,300000,300000,57
mean,1630.8213304110052,1637.557361027152,800.1971752921536,null,null,-0.0017583016983018318,null,265.74504333333334,null,1.6372379088620517,0.019203413940256035,307.65162659640646,null,null,5453132.203883333,7.285714285714286,8.0,null,null,null,8.0,null,2197.7893437320627,null,474.64413941001527,111.0,null,3.1930333817126223E90,322486.0776138271,0.2546199975523192,2785.918968223517,null,4193.803316943078,340.67465036934516,2059.3398745633426,2.321136666666667,null,2018.0,217.24561403508773
stddev,57.17088192225127,53.348946628936424,948.4930169775345,null,null,0.040913630819626894,null,46.160696807975754,null,0.30109540010821134,0.18639516482372667,36.00347398479321,null,null,3903158.3914881404,3.5326937743674747,0.0,null,null,null,0.0,null,517.5691469742331,null,5660.558288038367,128.17175976009693,null,3.349852913181436E91,683865.5731852937,0.4356607265530351,320.19677358346576,null,1161.510074269226,107.15502739584835,353.0583580218701,22.574023711554194,null,0.816497941758763,21.869973464738614
min,501,501,,,,-0.078,null,185,null,1.0,0.0,11,,,307,,,AT,,AA-IVA,,,0,F,,,,,,0,0,null,898,27,242,1,v15,2017,100
max,2895,2250,unknown,m1,m1,2.0,null,671,null,3.0,1.9,535,P,petrol/electric,15499148,e24*2,ZESZUTA,SK,VOLVO CAR CORPORATION,unknown,ZENVO,VW GROUP PC,4612,P,l50,e9*ks07/46*6716*01,RL-ZE1AEM57A6_0000-SJN-1,ZYX10(H),h,1,4762,null,8382,1103,4160,3250,v20,2019,296


In [0]:
display(df2)

At1 (mm),At2 (mm),Cn,Cr,Ct,De,E (g/km),Enedc (g/km),Er (g/km),Ernedc (g/km),Erwltp (g/km),Ewltp (g/km),Fm,Ft,ID,It,MMS,MS,Man,Mh,Mk,Mp,Mt,Status,T,TAN,VFN,Va,Ve,Vf,W (mm),Zr,ec (cm3),ep (KW),m (kg),r,version_file,year,z (Wh/km)
null,null,BUGATTI CHIRON-DIVO,M1,M1,null,null,575,null,null,null,null,M,petrol,4169522,,BUGATTI F,DE,AA-IVA,AA-IVA,BUGATTI,,null,P,,,,,,null,null,null,null,1103,1995,1,v19,2019,null
null,null,BUGATTI BG 744,M1,M1,null,null,575,null,null,null,null,M,petrol,4169823,,BUGATTI F,DE,AA-IVA,AA-IVA,BUGATTI,,null,F,,,,,,null,null,null,7993,1103,1995,1,v20,2019,null
null,null,BUGATTI BG 744,M1,M1,null,null,575,null,null,null,null,M,petrol,4169823,,BUGATTI F,DE,AA-IVA,AA-IVA,BUGATTI,,null,P,,,,,,null,null,null,7993,1103,1995,1,v19,2019,null
null,null,BUGATTI CHIRON-DIVO,M1,M1,null,null,575,null,null,null,null,M,petrol,4169522,,BUGATTI F,DE,AA-IVA,AA-IVA,BUGATTI,,null,F,,,,,,null,null,null,null,1103,1995,1,v20,2019,null
null,null,AUDI A8,M1,M1,null,null,545,null,null,null,null,M,petrol,4784577,,AUDI,DE,AA-IVA,AA-IVA,AUDI,,null,F,F8,,,,,null,null,null,3993,420,3895,1,v20,2019,null
null,null,AUDI A8,M1,M1,null,null,545,null,null,null,null,M,petrol,4784577,,AUDI,DE,AA-IVA,AA-IVA,AUDI,,null,P,F8,,,,,null,null,null,3993,420,3895,1,v19,2019,null
1750,1670,BUGATTI CHIRON,M1,M1,null,null,516,null,null,null,506,M,petrol,7958396,,BUGATTI AUTOMOBILES SAS,NL,AA-IVA,AA-IVA,BUGATTI,,2150,F,5B,e1*KS07/46*0008*09,,DALAADX1CN,TAD7AD71C002,null,2710,null,7993,1103,2070,1,v20,2019,null
1747,1670,,M1,M1,null,null,516,null,null,null,505,M,petrol,754447,,BUGATTI,GB,BUGATTI AUTOMOBILES SAS,BUGATTI,BUGATTI,VW GROUP PC,2150,P,5B,e1*ks07/46*0008*09,,DALAADX1CN,TAD7AD71C002,null,2711,null,7993,null,2070,1,v19,2019,null
1747,1670,BUGATTI CHIRON,M1,M1,null,null,516,null,null,null,null,M,petrol,7650625,,BUGATTI F,DE,BUGATTI AUTOMOBILES SAS,BUGATTI,BUGATTI,VW GROUP PC,null,F,5B,E1*KS07/46*0008*06,,DALAADX1CN,TAD7AD71C002,null,2711,null,7993,1103,2070,1,v20,2019,null
1747,1670,BUGATTI CHIRON,M1,M1,null,null,516,null,null,null,null,M,petrol,7650624,,BUGATTI F,DE,BUGATTI AUTOMOBILES SAS,BUGATTI,BUGATTI,VW GROUP PC,null,F,5B,E1*KS07/46*0008*06,,DALAADX1CN,TAD7AD71C002,null,2711,null,7993,1103,2070,1,v20,2019,null
